
### Question 1.1
The below 'add_ticker' function is acting weird. The number of element contained by the portfolio doesn't match what we would expect and the invested time neither.
Please fix the code and explain the reason of the strange behaviour.

In [ ]:

import time

### modify the function to fix the issues you find
def add_tickers(account, tickers, portfolio=[], invested_at=time.time()):
    portfolio.extend(tickers)
    time.sleep(3)
    print(f"""---- Added {len(tickers)} to the {account}. ----
    The portfolio contains now {len(portfolio)} different tickers.
    Invested since {round(time.time()-invested_at, 3)} seconds\n""")

    return portfolio
    

In [ ]:
from dataclasses import dataclass
# Sample data
@dataclass
class Ticker:
    symbol: str
    quantity: int
    price: float
    asset_class: str
    
interviewer_portfolio= [
    # Ticker('GOLD', 2, 20.91, 'FX'),
    # Ticker('SI=F', 2, 13.56, 'FX'),
    Ticker('SPY', 2, 5.56, 'ETF'),
    Ticker('VOO', -1, 25.21, 'ETF'),
    Ticker('MSFT', 13, 79.56, 'Equity'),
    Ticker('AAPL', -10, -23.5, 'Equity'),
    Ticker('AMZN', 4, -274.5, 'Equity'),
]

In [ ]:
pf_1 = add_tickers("Demo account", tickers=[
    Ticker('SPY', -25, 5.56, 'ETF'),
    Ticker('MSFT', 13, 3.76, 'Equity'),
    Ticker('FXAIX', -2, 16.24, 'Fund')
    ])

pf_2= add_tickers("Manager account", 
                  tickers=[ Ticker('ZL=F', 34, -23.5, 'Future')
                           ])

pf_3 = add_tickers("Interviewer's account", tickers=[
    Ticker('ETH-USD', 10, 1256.56, 'CCC'),
    Ticker('BTC-USD', 1, 15_000.56, 'CCC '),
    Ticker('AAPL', 12, 143.43, 'Equity')],

    portfolio=interviewer_portfolio)


print(pf_2)
print(pf_3)

### Question 1.2
How would you find the *asset classes* present in *portfolio 1* that are not present in the *portfolio 3*? Propose a solution:

In [ ]:
### Write your solution here

### Question 1.3:
we need to know which positions are short (negative quantity) in our *portfolio 1* for the classes **Equity** and **ETF**. How would you achieve that?

In [ ]:
### Write your solution here

### Question 1.4: (optional)
For a given portfolio, would you be able to provide a solution to count the number of tickers present on each *asset class* category?

In [ ]:
### Write your solution here

## Code Review

### Question 2.1
Look at the code below and express your opinion on its structure. What are your considerations? If you had to refactor it, which actions would you take?

In [ ]:
from dataclasses import dataclass

ROLES = ['demo', 'default', 'admin']
        
@dataclass
class User:
    name: str
    role: str
    
    def get_positions(self):
        if self.role=='admin':
            print( 'retrieving all the positions available in the db')
        else:
            if self.role=='default':
                print(f'retrieving only the positions in which {self.name} user is invested')
            elif self.role=='demo':
                print('retrieving only public available tickers')
            else:
                print(f"you don't have a valid role. The role should be one of [{', '.join(str(role) for role in ROLES)}]")


prospect = User('Richmond', 'demo')
client = User('Smart Guy', 'default')
it_guy = User('Maurice Moss', 'admin')
manager = User('Jen Barber', 'manager')


prospect.get_positions()
client.get_positions()
it_guy.get_positions()
manager.get_positions()

### Question 2.2
Look at the code below and express your opinion on its structure. What are your considerations? If you had to refactor it, which actions would you take? Has this the same issues as the above script?

In [79]:
import yfinance as yf
import pandas as pd

class portfolio():
    
    def __init__(self, tickers= None):
        self.tickers = tickers or  ['0005.HK', '0006.HK', '0066.HK', '0700.HK', '2800.HK']
        self.df = pd.DataFrame()
        print('initialized')

    
    def download_prices(self, benchmarks= None):
        benchmarks= benchmarks or []
        stock_list = self.tickers
        data = yf.download(stock_list, start="2019-06-01", end="2020-02-21")
        print('data fields downloaded:', set(data.columns.get_level_values(0)))
        self.df = data.Close
        return self.df
    
    def performances(self, frequency='Monthly'):
        
        try: 
            if frequency=='Daily':
                print('computing Daily perfs')
                perf= self.df.pct_change()
            elif frequency=='Monthly':
                print('computing Monhtly perfs')
                perf=  self.df.sort_index().resample("M").apply(lambda x : x.iloc[-1]).pct_change()
            elif frequency == 'Weekly':
                print('computing Weekly perfs')
                perf=  self.df.sort_index().resample('W-MON', label='left', closed='left').apply(lambda x : x.iloc[-1]).pct_change()
            else:
                raise ValueError('the frequency requested is not available')
        except:
            print("couldn't compute preformances")
        return perf

my_portfolio= portfolio(['GOLD', 'SI=F','AMZN','MSFT','AAPL'] )
my_portfolio.download_prices()
my_portfolio.df.drop('SI=F', axis=1)



initialized
[*********************100%***********************]  5 of 5 completed
data fields downloaded: {'High', 'Low', 'Open', 'Volume', 'Adj Close', 'Close'}


,AAPL,AMZN,GOLD,MSFT
Date,,,,
2019-06-03,43.325001,84.634499,13.090000,119.839996
2019-06-04,44.910000,86.477997,13.210000,123.160004
2019-06-05,45.634998,86.925003,13.230000,125.830002
2019-06-06,46.305000,87.718002,13.510000,127.820000
2019-06-07,47.537498,90.201500,13.540000,131.399994
...,...,...,...,...
2020-02-13,81.217499,107.493500,18.930000,183.710007
2020-02-14,81.237503,106.743500,19.719999,185.350006
2020-02-18,79.750000,107.783501,20.580000,187.229996


,AAPL,AMZN,GOLD,MSFT,SI=F
Date,,,,,
2019-06-03,43.325001,84.634499,13.090000,119.839996,14.703000
2019-06-04,44.910000,86.477997,13.210000,123.160004,14.732000
2019-06-05,45.634998,86.925003,13.230000,125.830002,14.754000
2019-06-06,46.305000,87.718002,13.510000,127.820000,14.868000
2019-06-07,47.537498,90.201500,13.540000,131.399994,14.994000
...,...,...,...,...,...
2020-02-13,81.217499,107.493500,18.930000,183.710007,17.600000
2020-02-14,81.237503,106.743500,19.719999,185.350006,17.715000
2020-02-18,79.750000,107.783501,20.580000,187.229996,18.131001


,AAPL,AMZN,GOLD,MSFT
Date,,,,
2019-06-03,43.325001,84.634499,13.090000,119.839996
2019-06-04,44.910000,86.477997,13.210000,123.160004
2019-06-05,45.634998,86.925003,13.230000,125.830002
2019-06-06,46.305000,87.718002,13.510000,127.820000
2019-06-07,47.537498,90.201500,13.540000,131.399994
...,...,...,...,...
2020-02-13,81.217499,107.493500,18.930000,183.710007
2020-02-14,81.237503,106.743500,19.719999,185.350006
2020-02-18,79.750000,107.783501,20.580000,187.229996
